In [ ]:
%run oeai_py

In [ ]:
# Create an instance of OEAI class and set the platform ("Synapse" or "Fabric")
oeai = OEAI("Fabric")

In [ ]:
# CHANGE VALUES FOR YOUR KEY VAULT
keyvault = "INSERT_YOUR_KEYVAULT_NAME_HERE" # Fabric requires full URL eg "https://key_vault_name.vault.azure.net/"
keyvault_linked_service = "INSERT_YOUR_LINKED_SERVICE_NAME_HERE"  # Not required for Fabric.

# Synapse OEA environment paths
silver_path = oeai.get_secret(spark, "msgraph-silver", keyvault_linked_service, keyvault)
silver_path_wonde = oeai.get_secret(spark, "wonde-silver", keyvault_linked_service, keyvault)
gold_path = oeai.get_secret(spark, "gold-path", keyvault_linked_service, keyvault)
storage_account_name = oeai.get_secret(spark, "storage-account", keyvault_linked_service, keyvault)
storage_account_access_key = oeai.get_secret(spark, "storage-accesskey", keyvault_linked_service, keyvault)

In [ ]:
def process_delta_tables_to_parquet(spark, storage_account_name, storage_account_access_key, silver_path, gold_path):
    """
    Sets up configuration for Azure storage access, lists subdirectories in the silver path, and processes
    each Delta Lake table by converting and saving it in Parquet format in the gold path.

    Args:
        spark (SparkSession): Active Spark session.
        storage_account_name (str): Azure storage account name.
        storage_account_access_key (str): Access key for the Azure storage account.
        silver_path (str): Path to the silver layer directory (source Delta tables).
        gold_path (str): Path to the gold layer directory (destination for Parquet files).

    This function will process each Delta Lake table found in the silver layer, partition the data by 
    'organisationkey', and write it to the gold layer as Parquet files.
    """
    # Set up the configuration for accessing the storage account
    spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net", storage_account_access_key)

    sc = spark.sparkContext
    hadoop_conf = sc._jsc.hadoopConfiguration()
    hadoop_conf.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
    hadoop_conf.set("fs.azure.account.key." + storage_account_name + ".dfs.core.windows.net", storage_account_access_key)

    # URI for the parent directory
    parent_dir_uri = sc._gateway.jvm.java.net.URI(silver_path)

    # Hadoop Path of the parent directory
    Path = sc._gateway.jvm.org.apache.hadoop.fs.Path
    FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem

    # Get the FileSystem for the given URI and configuration
    fs = FileSystem.get(parent_dir_uri, hadoop_conf)

    # List the subdirectories at the given URI
    status = fs.listStatus(Path(silver_path))
    delta_table_paths = [file.getPath().toString() for file in status if file.isDirectory()]

    for table_path in delta_table_paths:
        try:
            df = spark.read.format("delta").load(table_path)
            table_name = os.path.basename(urlparse(table_path).path)
            parquet_output_folder_path = os.path.join(gold_path, table_name)
            df.write.mode("overwrite").format("parquet").save(parquet_output_folder_path)
            
            
        except AnalysisException as e:
            print(f"Error reading Delta table at {table_path}: ", e)

In [ ]:
# Initialize Spark session
process_delta_tables_to_parquet(spark, storage_account_name, storage_account_access_key, silver_path, gold_path)

In [ ]:
fact_graph_ReadingProgress_path = f"{silver_path}/fact_graph_ReadingProgress"
dim_graph_Assignment_path = f"{silver_path}/dim_graph_Assignment"
dim_graph_User_path = f"{silver_path}/dim_graph_User"
dim_Student_path = f"{silver_path_wonde}/dim_Student"
dim_Organisation_path = f"{silver_path_wonde}/dim_Organisation"

# Load Delta tables into DataFrames
df_fact_graph_ReadingProgress = spark.read.format("delta").load(fact_graph_ReadingProgress_path)
df_dim_graph_Assignment = spark.read.format("delta").load(dim_graph_Assignment_path)
df_dim_graph_User = spark.read.format("delta").load(dim_graph_User_path)
df_dim_Student = spark.read.format("delta").load(dim_Student_path)
df_dim_Organisation = spark.read.format("delta").load(dim_Organisation_path)

In [ ]:
# Join fact_graph_ReadingProgress with dim_graph_Assignment
df_enriched = df_fact_graph_ReadingProgress.join(
    df_dim_graph_Assignment,
    df_fact_graph_ReadingProgress['assignmentid'] == df_dim_graph_Assignment['external_id'],
    how='left'
)

# Join the result with dim_graph_User
df_enriched = df_enriched.join(
    df_dim_graph_User,
    df_enriched['studentid'] == df_dim_graph_User['external_id'],
    how='left'
)

# Join the result with dim_Student using studentkey from dim_graph_User
df_enriched = df_enriched.join(
    df_dim_Student,
    df_dim_graph_User['studentkey'] == df_dim_Student['studentkey'],
    how='left'
)

# Join the result with dim_Organisation using organisationkey from dim_Student
df_enriched = df_enriched.join(
    df_dim_Organisation,
    df_dim_Student['organisationkey'] == df_dim_Organisation['organisationkey'],
    how='left'
)



In [ ]:
from pyspark.sql.functions import col

# Select and rename the columns
df_enriched = df_enriched.select(
    df_fact_graph_ReadingProgress['*'],
    df_dim_graph_Assignment['displayName'],
    df_dim_Organisation['Organisation_Name']
)


In [ ]:
# Write the result to Parquet
df_enriched.write.format("parquet").mode("overwrite").save(gold_path + 'fact_graph_ReadingProgress')

print(f"Enriched fact_graph_ReadingProgress has been saved to Parquet at {gold_path}")
